<a href="https://colab.research.google.com/github/Gagalin105/PL-groupProject/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E5%B0%88%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **功能簡述**
## 1. 細分項目記帳
#### 細項：
##### (1) 最大類別：ACG、偶像/樂團/球員/VTuber等等
##### (2) 第二類別：該對象/團體出身國家(ex：韓國、日本等等)
### *#(1)和(2)順序要對調？*
##### (3) 第三類別：周邊類別
→ex：立牌、徽章、同人本、CD、插畫集等等，可以自己增加類別
##### (4) 第四類別：日期&時間

##### *可加入功能：外幣匯率轉換*
## 2. 分析財務&設定預算
##### (1)依照選擇的類別生成圖表
→可以依照團體or作品、月份、或是周邊類別做比較，*希望可以做到多類別篩選*
##### (2)設定這個月的上限額度
→若離上限剩(500元or自行設定額度)會提醒
## 3. 查詢功能
##### 購買日期or作品or類別
## 4. AI建議
##### 財務警示&建議購買的收納物

# **寸得育部分**



In [ ]:
# 導入函式庫
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os

# 解決 Colab 中文顯示問題
!pip install -q chineseize-matplotlib
import chineseize_matplotlib

# 檔案儲存路徑
FILE_PATH = 'peripheral_accounting_data.csv'

# 全域預算設定
CURRENT_MONTH_BUDGET = 5000  # 總預算 (台幣)
WARNING_THRESHOLD = 500      # 警示閾值 (剩餘金額少於此值時發出警告)

def load_data():
    """載入或初始化 DataFrame。"""
    if os.path.exists(FILE_PATH):
        df = pd.read_csv(FILE_PATH)
        df['日期'] = pd.to_datetime(df['日期'])
    else:
        print(f"找不到 {FILE_PATH}，建立新的空資料表。")
        df = pd.DataFrame(columns=['日期', 'IP類別', '國家', '周邊類別', '金額', '幣別', '台幣金額'])
        df['日期'] = pd.to_datetime(df['日期'])
    return df

def save_data(df):
    """儲存 DataFrame 到 CSV 檔案。"""
    df.to_csv(FILE_PATH, index=False)
    print(f"\n✅ 資料已儲存至 {FILE_PATH}")

# 載入資料
df_records = load_data()
if not df_records.empty:
    df_records['月份'] = df_records['日期'].dt.to_period('M')

找不到 peripheral_accounting_data.csv，建立新的空資料表。


In [ ]:
def add_new_record_for_test(df):
    """
    簡化的記帳功能，用於快速新增測試數據。
    """
    print("\n--- 新增一筆測試記錄 ---")

    # 假設所有金額都已換算為台幣

    # 獲取輸入
    ip = input("IP/作品類別 (ex: VTuber): ")
    country = input("國家 (ex: 日本): ")
    item_type = input("周邊類別 (ex: 立牌): ")
    amount = float(input("台幣金額: "))

    # 設置日期 (使用當前日期)
    purchase_date = datetime.now().strftime('%Y-%m-%d')

    # 創建新行
    new_row = pd.DataFrame([{
        '日期': pd.to_datetime(purchase_date),
        'IP類別': ip,
        '國家': country,
        '周邊類別': item_type,
        '金額': amount,
        '幣別': 'TWD',
        '台幣金額': amount
    }])

    # 將新資料加入 DataFrame
    df = pd.concat([df, new_row], ignore_index=True)
    df['月份'] = df['日期'].dt.to_period('M')

    print("--- 記錄新增成功 ---")

    # 新增記錄後立即進行預算檢查
    check_budget(df, new_expense=amount)

    return df

In [ ]:
## 2.1 財務分析與圖表生成
def analyze_and_plot(df):
    """
    依照指定類別（IP、國家、周邊類別、月份）生成總金額統計圖表。
    """
    if df.empty:
        print("💡 尚無交易紀錄，無法進行分析。")
        return

    print("\n--- 財務分析與圖表生成 ---")
    print("請選擇分析的維度:")
    print("1. IP/作品類別")
    print("2. 國家")
    print("3. 周邊類別")
    print("4. 月份")

    choice = input("請輸入號碼 (1/2/3/4): ")

    category_map = {
        '1': 'IP類別', '2': '國家', '3': '周邊類別', '4': '月份'
    }

    if choice not in category_map:
        print("輸入無效。")
        return

    analysis_category = category_map[choice]

    # 1. 依照類別分組加總金額
    summary = df.groupby(analysis_category)['台幣金額'].sum().sort_values(ascending=False)

    # 2. 生成圖表
    plt.figure(figsize=(10, 6))
    summary.plot(kind='bar', color='darkcyan')
    plt.title(f'周邊支出 - 依 {analysis_category} 區分', fontsize=16)
    plt.ylabel('總金額 (NT$)', fontsize=12)
    plt.xlabel(analysis_category, fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', linestyle='--')
    plt.tight_layout()
    plt.show()
    # 觸發一個圖表範例
    #

    print(f"\n--- 依 {analysis_category} 區分的支出總結 ---")
    print(summary.to_string(float_format="NT$ {:,.0f}".format))


In [ ]:
## 2.2 設定預算與警示
def check_budget(df, new_expense=0):
    """檢查當月預算並發出警示。"""
    # 檢查 DataFrame 是否為空
    if df.empty:
        if new_expense > 0:
            print(f"\n--- 預算檢查報告 ---")
            remaining = CURRENT_MONTH_BUDGET - new_expense
            print(f"🗓️ 當月預算總額: NT$ {CURRENT_MONTH_BUDGET:,.0f}")
            print(f"💰 當前總支出: NT$ {new_expense:,.0f}")
            print(f"⏳ 剩餘額度: NT$ {remaining:,.0f}")
            return remaining >= 0
        else:
            print("💡 尚無交易紀錄，無需檢查預算。")
            return

    # 獲取當前月份
    current_month_str = datetime.now().strftime('%Y-%m')
    current_month = pd.to_datetime(current_month_str).to_period('M')
    if '月份' not in df.columns:
         df['月份'] = df['日期'].dt.to_period('M')

    # 篩選當月資料
    monthly_df = df[df['月份'] == current_month]

    # 計算目前總支出 (包含本次新增的費用)
    current_spending = monthly_df['台幣金額'].sum() + new_expense

    remaining = CURRENT_MONTH_BUDGET - current_spending

    print("\n--- 預算檢查報告 ---")
    print(f"🗓️ {current_month_str} 月預算總額: NT$ {CURRENT_MONTH_BUDGET:,.0f}")
    print(f"💰 當前總支出: NT$ {current_spending:,.0f}")
    print(f"⏳ 剩餘額度: NT$ {remaining:,.0f}")

    if remaining < 0:
        print(f"🚨 **嚴重超支警示！** 已超支 NT$ {-remaining:,.0f}，請立即調整消費習慣。")
        return False
    elif remaining <= WARNING_THRESHOLD:
        print(f"⚠️ **預算警示！** 剩餘額度只剩 NT$ {remaining:,.0f}，請節制！")
        return True
    else:
        print("✅ 預算狀況良好，請繼續維持。")
        return True

In [ ]:
## 3 查詢功能
def query_records(df):
    """
    提供一個互動式介面，讓使用者可以依照日期、作品/IP或類別進行查詢。
    """
    if df.empty:
        print("💡 尚無交易紀錄，無法進行查詢。")
        return

    print("\n--- 交易紀錄查詢功能 ---")
    print("請選擇查詢方式：")
    print("1. 依日期/月份 (格式：YYYY-MM-DD 或 YYYY-MM)")
    print("2. 依 IP/作品 (例如：VTuber 名稱、動畫名)")
    print("3. 依 周邊類別 (例如：立牌、徽章)")

    choice = input("請輸入號碼 (1/2/3): ")

    if choice == '1':
        query_type = '日期'
        keyword = input("請輸入查詢日期/月份: ")
        # 日期查詢：將日期欄位格式化為字串進行模糊匹配
        result_df = df[df['日期'].dt.strftime('%Y-%m-%d').str.contains(keyword, na=False)]
    elif choice == '2':
        query_type = 'IP類別'
        keyword = input("請輸入 IP/作品 關鍵字: ")
        # 關鍵字查詢：使用 str.contains 進行模糊匹配 (不區分大小寫)
        result_df = df[df['IP類別'].str.contains(keyword, case=False, na=False)]
    elif choice == '3':
        query_type = '周邊類別'
        keyword = input("請輸入 周邊類別 關鍵字: ")
        result_df = df[df['周邊類別'].str.contains(keyword, case=False, na=False)]
    else:
        print("輸入無效。")
        return

    print(f"\n--- 查詢結果 (篩選條件: {query_type} 包含 '{keyword}') ---")

    if result_df.empty:
        print("未找到符合條件的紀錄。")
    else:
        # 顯示結果並格式化輸出，只顯示關鍵欄位
        display_cols = ['日期', 'IP類別', '國家', '周邊類別', '台幣金額']
        print(result_df[display_cols].to_string(index=False, float_format="NT$ {:,.0f}".format))
        print(f"\n總計筆數: {len(result_df)} 筆, 總金額: NT$ {result_df['台幣金額'].sum():,.0f}")

In [ ]:
## 4 AI建議
def ai_suggestion(df):
    """
    提供財務警示和收納建議。
    """
    print("\n--- AI 財務與收納建議 ---")

    # 4.1 財務警示
    print("#### 財務行為分析 ####")
    if df.empty:
        print("💡 紀錄不足，無法提供財務行為分析。")
        return

    # 計算單一 IP 佔總支出的比例
    total_spending = df['台幣金額'].sum()
    ip_summary = df.groupby('IP類別')['台幣金額'].sum().sort_values(ascending=False)

    if not ip_summary.empty:
        # 支出最高的 IP
        highest_ip = ip_summary.index[0]
        highest_amount = ip_summary.iloc[0]
        percentage = (highest_amount / total_spending) * 100

        print(f"💸 您最大的支出集中在 **{highest_ip}** (NT$ {highest_amount:,.0f})，佔總支出的 **{percentage:.1f}%**。")

        if percentage > 50:
             print("🚨 警示：單一目標支出佔比過高，請留意風險分散，並確保財務平衡。")

    # 4.2 收納建議
    print("\n#### 周邊收納建議 ####")

    # 檢查購買數量最多的周邊類別
    item_counts = df['周邊類別'].value_counts()

    if not item_counts.empty:
        most_bought_item = item_counts.index[0]

        advice = "建議：通用收納箱、防潮包。"

        if '徽章' in most_bought_item or '小卡' in most_bought_item:
            advice = "建議購買：**A4或B5活頁收納冊**、卡套、磁吸夾層板，保持乾燥。"
        elif '立牌' in most_bought_item or '吊飾' in most_bought_item:
            advice = "建議購買：**分層壓克力展示架**、防塵透明盒，避免陽光直射。"
        elif '同人本' in most_bought_item or '插畫集' in most_bought_item:
            advice = "建議購買：**防酸書套**、直立式收納盒，避免側壓造成裝幀變形。"

        print(f"📦 您最近常購買 **{most_bought_item}**。")
        print(f"👉 {advice}")

In [ ]:
def main_menu(df):
    """主程式選單。"""

    while True:
        print("\n" + "="*40)
        print("        💰 周邊商品記帳分析系統 💰")
        print("="*40)
        print("1. 📓 新增一筆記錄 (測試用)")
        print("2. 📊 財務分析與圖表生成")
        print("3. 🔍 查詢歷史記錄")
        print("4. 💡 AI 建議 (財務警示與收納)")
        print("5. 🚨 查看當月預算狀況")
        print("6. 💾 儲存並離開")

        choice = input("請選擇功能號碼 (1-6): ")

        if choice == '1':
            df = add_new_record_for_test(df)
        elif choice == '2':
            analyze_and_plot(df)
        elif choice == '3':
            query_records(df)
        elif choice == '4':
            ai_suggestion(df)
        elif choice == '5':
            check_budget(df)
        elif choice == '6':
            save_data(df)
            print("系統已結束，期待下次使用！")
            break
        else:
            print("輸入錯誤，請輸入 1 到 6 之間的數字。")

In [ ]:
# 執行主選單
main_menu(df_records)


        💰 周邊商品記帳分析系統 💰
1. 📓 新增一筆記錄 (測試用)
2. 📊 財務分析與圖表生成
3. 🔍 查詢歷史記錄
4. 💡 AI 建議 (財務警示與收納)
5. 🚨 查看當月預算狀況
6. 💾 儲存並離開


# Task
```python
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def get_google_sheet(credentials_path, spreadsheet_name, worksheet_index=0):
    """
    連線到 Google Sheet 並回傳指定的 worksheet 物件。

    Args:
        credentials_path (str): 服務帳號金鑰 (JSON 檔案) 的路徑。
        spreadsheet_name (str): Google Sheet 的名稱。
        worksheet_index (int): 要操作的工作表索引 (預設為第一個工作表，索引為 0)。

    Returns:
        gspread.worksheet.Worksheet: 可操作的 Google Sheet 工作表物件。
    """
    try:
        # 定義 Google Sheets API 的範圍
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

        # 使用服務帳號金鑰進行認證
        creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
        client = gspread.authorize(creds)

        # 打開指定的 Google Sheet
        spreadsheet = client.open(spreadsheet_name)

        # 取得指定索引的工作表
        worksheet = spreadsheet.get_worksheet(worksheet_index)

        print(f"✅ 成功連線到 Google Sheet: '{spreadsheet_name}', 工作表: '{worksheet.title}'")
        return worksheet

    except Exception as e:
        print(f"❌ 連線到 Google Sheet 時發生錯誤: {e}")
        print("請確認以下事項：")
        print("1. 服務帳號金鑰檔案路徑是否正確？")
        print("2. Google Cloud Project 中是否已啟用 Google Drive API？")
        print("3. Google Sheet 是否已分享給您的服務帳號 (給予編輯者權限)？")
        print("4. Google Sheet 名稱是否正確？")
        return None

# 範例用法 (這部分將在後續的儲存格中實際執行)
# credentials_path = 'your-service-account-key.json'
# spreadsheet_name = 'Peripheral Accounting Data'
# worksheet = get_google_sheet(credentials_path, spreadsheet_name)
# if worksheet:
#     # 你現在可以使用 worksheet 物件來讀取或寫入資料
#     # 例如：worksheet.append_row(['新資料1', '新資料2'])
#     pass
```

# **Gaga**

In [ ]:
get_ipython().run_line_magic('pip', 'install gspread oauth2client')
get_ipython().run_line_magic('pip', 'install gradio')
print("Successfully installed gspread and oauth2client.")

In [ ]:
import gradio as gr
import datetime
import os
import json

def get_google_sheet(credentials_path, spreadsheet_name, worksheet_index=0):
    import gspread
    from oauth2client.service_account import ServiceAccountCredentials
    try:
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
        client = gspread.authorize(creds)
        spreadsheet = client.open(spreadsheet_name)
        worksheet = spreadsheet.get_worksheet(worksheet_index)
        print(f"成功連線到 Google Sheet: '{spreadsheet_name}', 工作表: '{worksheet.title}'")
        return worksheet, spreadsheet.id
    except Exception as e:
        print(f"連線到 Google Sheet 時發生錯誤: {e}")
        return None, None

def convert_currency(amount, currency):
    exchange_rates = {
        'USD': 30.5,
        'JPY': 0.22,
        'KRW': 0.023,
        'EUR': 33.0,
        'TWD': 1.0
    }
    if currency.upper() in exchange_rates:
        twd_amount = amount * exchange_rates[currency.upper()]
        print(f"轉換結果: {amount} {currency.upper()} = {twd_amount:.2f} TWD")
        return twd_amount
    else:
        print(f"警告: 不支援的幣別 '{currency}'。請手動輸入台幣金額或更新匯率設定。")
        return amount

def accountingInCategory(ip_category, country, ip_name, item_type, amount, currency, purchase_date, purchase_time, worksheet=None):
  twd_amount = convert_currency(amount, currency)
  record_data = {
      '日期': purchase_date,
      'IP類別': ip_category,
      '國家': country,
      'IP名稱': ip_name,
      '周邊類別': item_type,
      '金額': amount,
      '幣別': currency.upper(),
      '台幣金額': twd_amount,
      '時間': purchase_time
  }
  print(f"\n已準備好以下記錄：{record_data}")
  if worksheet:
      headers = ['日期', 'IP類別', '國家', 'IP名稱', '周邊類別', '金額', '幣別', '台幣金額', '時間']
      if not worksheet.get_all_values():
          worksheet.append_row(headers)
          print("Google Sheet 標頭已寫入。")
      row_data = [
          record_data['日期'],
          record_data['IP類別'],
          record_data['國家'],
          record_data['IP名稱'],
          record_data['周邊類別'],
          record_data['金額'],
          record_data['幣別'],
          record_data['台幣金額'],
          record_data['時間']
      ]
      worksheet.append_row(row_data)
      print("資料已成功寫入 Google Sheet。")
  return record_data

def process_accounting_data(ip_category, country, ip_name, item_type, amount, currency, purchase_date, purchase_time, credentials_file, spreadsheet_name):
    if credentials_file is None:
        return "錯誤：請上傳服務帳號金鑰檔案。"
    if not spreadsheet_name:
        return "錯誤：請輸入 Google Sheet 名稱。"
    credentials_path = credentials_file.name

    try:
        worksheet, spreadsheet_id = get_google_sheet(credentials_path, spreadsheet_name)

        if worksheet is None or spreadsheet_id is None:
            return "無法連線到 Google Sheet。請檢查金鑰檔案和 Sheet 名稱。"
        result = accountingInCategory(
            ip_category=ip_category,
            country=country,
            ip_name=ip_name,
            item_type=item_type,
            amount=amount,
            currency=currency,
            purchase_date=purchase_date,
            purchase_time=purchase_time,
            worksheet=worksheet
        )
        google_sheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/edit"
        success_message = (
            f"記帳成功！已新增記錄：{result['IP類別']} - {result['IP名稱']} - {result['周邊類別']}，"\
            f"台幣金額：{result['台幣金額']:.2f}<br>"
            f"<a href='{google_sheet_url}' target='_blank'>點擊此處查看您的 Google Sheet</a>"
        )
        return success_message

    except Exception as e:
        return f"處理記帳時發生錯誤：{e}"
ip_category_input = gr.Textbox(label="IP類別 (例如: VTuber)")
country_input = gr.Textbox(label="國家 (例如: 日本)")
ip_name_input = gr.Textbox(label="IP名稱 (例如: Mio、Hololive)")
item_type_input = gr.Textbox(label="周邊類別 (例如: 徽章)")
amount_input = gr.Number(label="金額")
currency_input = gr.Dropdown(choices=["TWD", "USD", "JPY", "KRW", "EUR"], label="幣別")
purchase_date_input = gr.Textbox(label="購買日期 (YYYY-MM-DD)", value=datetime.date.today().strftime("%Y-%m-%d"))
purchase_time_input = gr.Textbox(label="購買時間 (HH:MM:SS)", value=datetime.datetime.now().strftime("%H:%M:%S"))
credentials_file_input = gr.File(label="服務帳號金鑰檔案 (.json)", file_types=[".json"])
spreadsheet_name_input = gr.Textbox(label="Google Sheet 名稱")

interface = gr.Interface(
    fn=process_accounting_data,
    inputs=[
        ip_category_input,
        country_input,
        ip_name_input,
        item_type_input,
        amount_input,
        currency_input,
        purchase_date_input,
        purchase_time_input,
        credentials_file_input,
        spreadsheet_name_input
    ],
    outputs=gr.HTML(label="操作結果"),
    title="周邊商品記帳系統",
    flagging_mode="manual"
)
interface.launch()

# 整合

In [6]:
!pip install -q chineseize-matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 63.0 MB/s eta 0:00:00


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
COLAB_CREDENTIALS_PATH = '/content/drive/MyDrive/key/gen-lang-client-0600039359-89d29e6382b8.json'

In [19]:
import gradio as gr
import datetime
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import matplotlib.pyplot as plt
import chineseize_matplotlib
import io
import os
import json
from PIL import Image

CURRENT_MONTH_BUDGET = 5000
WARNING_THRESHOLD = 500
EXPECTED_SHEET_HEADERS = ['日期', 'IP類別', '國家', 'IP名稱', '周邊類別', '金額', '幣別', '台幣金額', '時間']

def get_google_sheet(credentials_path, spreadsheet_name, worksheet_index=0):
    try:
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
        client = gspread.authorize(creds)
        spreadsheet = client.open(spreadsheet_name)
        worksheet = spreadsheet.get_worksheet(worksheet_index)
        print(f"成功連線到 Google Sheet: '{spreadsheet_name}', 工作表: '{worksheet.title}'")
        return worksheet, spreadsheet.id
    except Exception as e:
        print(f"連線到 Google Sheet 時發生錯誤: {e}")
        return None, None

def read_google_sheet_to_dataframe(credentials_path, spreadsheet_name):
    worksheet, _ = get_google_sheet(credentials_path, spreadsheet_name)
    if worksheet is None:
        return pd.DataFrame(columns=EXPECTED_SHEET_HEADERS), [], [], [], [] # Return empty lists for categories

    data = worksheet.get_all_values()
    if not data or not data[0]:
        worksheet.append_row(EXPECTED_SHEET_HEADERS)
        print("Google Sheet 標頭已插入。")
        data = worksheet.get_all_values()
    elif data[0] != EXPECTED_SHEET_HEADERS:
        # Check if headers are just reordered or completely different
        current_headers = data[0]
        if set(current_headers) != set(EXPECTED_SHEET_HEADERS):
            # If headers are significantly different, log a warning or attempt to re-align
            print("警告: Google Sheet 標頭與預期不符。將嘗試使用預期標頭讀取資料。")
        # Ensure the expected headers are at the top, if not, insert them
        if current_headers != EXPECTED_SHEET_HEADERS:
            try:
                # Find existing headers row and delete, then insert, or just insert if absent
                # A more robust solution might involve validating and potentially reordering columns if data exists
                # For simplicity here, we assume if current_headers != EXPECTED_SHEET_HEADERS,
                # we'll just insert the expected ones assuming the data below them is ordered correctly.
                # A more robust approach might be needed if column order changes.
                pass # Already handled by pandas later using column names
            except Exception as e:
                print(f"嘗試調整 Google Sheet 標頭時發生錯誤: {e}")

    if not data or len(data) < 1: # data can be empty or only header
        print("Google Sheet 中沒有足夠的資料可以讀取 (只有標頭或空白)。")
        return pd.DataFrame(columns=EXPECTED_SHEET_HEADERS), [], [], [], []

    # Use the first row as headers, and the rest as data
    df = pd.DataFrame(data[1:], columns=data[0])

    # Ensure all expected headers are present, add if missing
    for header in EXPECTED_SHEET_HEADERS:
        if header not in df.columns:
            df[header] = None # Add missing columns with None

    # Filter to only expected columns and reorder them
    df = df[EXPECTED_SHEET_HEADERS]

    if '日期' in df.columns:
        df['日期'] = pd.to_datetime(df['日期'], errors='coerce')
        df['月份'] = df['日期'].dt.to_period('M')
    if '台幣金額' in df.columns:
        df['台幣金額'] = pd.to_numeric(df['台幣金額'], errors='coerce')
    df.dropna(subset=['日期', '台幣金額'], inplace=True)

    print("\nGoogleSheet讀取的DataFrame資訊")
    print("DataFrame Columns:", df.columns.tolist())
    print("DataFrame Head:\n", df.head())
    print("-------------------------------------------")

    # Extract and sort unique values for dropdowns
    unique_ip_categories = sorted(df['IP類別'].dropna().unique().tolist()) if 'IP類別' in df.columns else []
    unique_countries = sorted(df['國家'].dropna().unique().tolist()) if '國家' in df.columns else []
    unique_ip_names = sorted(df['IP名稱'].dropna().unique().tolist()) if 'IP名稱' in df.columns else []
    unique_item_types = sorted(df['周邊類別'].dropna().unique().tolist()) if '周邊類別' in df.columns else []

    return df, unique_ip_categories, unique_countries, unique_ip_names, unique_item_types

def convert_currency(amount, currency):
    exchange_rates = {
        'USD': 30.5,
        'JPY': 0.22,
        'KRW': 0.023,
        'EUR': 33.0,
        'TWD': 1.0
    }
    if currency.upper() in exchange_rates:
        twd_amount = amount * exchange_rates[currency.upper()]
        print(f"轉換結果: {amount} {currency.upper()} = {twd_amount:.2f} TWD")
        return twd_amount
    else:
        print(f"警告: 不支援的幣別 '{currency}'。請手動輸入台幣金額或更新匯率設定。")
        return amount

def accountingInCategory(ip_category, country, ip_name, item_type, amount, currency, purchase_date, purchase_time, worksheet=None):
    twd_amount = convert_currency(amount, currency)
    record_data = {
        '日期': purchase_date,
        'IP類別': ip_category,
        '國家': country,
        'IP名稱': ip_name,
        '周邊類別': item_type,
        '金額': amount,
        '幣別': currency.upper(),
        '台幣金額': twd_amount,
        '時間': purchase_time
    }
    print(f"\n已準備好以下記錄：{record_data}")
    if worksheet:
        row_data = [
            record_data.get(header, '') for header in EXPECTED_SHEET_HEADERS # Use .get with default empty string
        ]
        worksheet.append_row(row_data)
        print("資料已成功寫入 Google Sheet。")
    return record_data

def check_budget(df, new_expense=0):
    if df.empty:
        if new_expense > 0:
            print(f"\n預算檢查報告：")
            remaining = CURRENT_MONTH_BUDGET - new_expense
            print(f"當月預算總額: NT$ {CURRENT_MONTH_BUDGET:,.0f}")
            print(f"當前總支出: NT$ {new_expense:,.0f}")
            print(f"剩餘額度: NT$ {remaining:,.0f}")
            return remaining >= 0
        else:
            print("尚無交易紀錄，無需檢查預算。")
            return
    current_month_str = datetime.datetime.now().strftime('%Y-%m')
    current_month = pd.to_datetime(current_month_str).to_period('M')
    if '月份' not in df.columns:
         df['月份'] = df['日期'].dt.to_period('M')
    monthly_df = df[df['月份'] == current_month]
    current_spending = monthly_df['台幣金額'].sum() + new_expense

    remaining = CURRENT_MONTH_BUDGET - current_spending

    print("\n預算檢查報告：")
    print(f"{current_month_str} 月預算總額: NT$ {CURRENT_MONTH_BUDGET:,.0f}")
    print(f"當前總支出: NT$ {current_spending:,.0f}")
    print(f"剩餘額度: NT$ {remaining:,.0f}")

    if remaining < 0:
        print(f"[嚴重超支警示！]已超支 NT$ {-remaining:,.0f}，請立即調整消費習慣。")
        return False
    elif remaining <= WARNING_THRESHOLD:
        print(f"[預算警示！]剩餘額度只剩 NT$ {remaining:,.0f}，請節制！")
        return True
    else:
        print("預算狀況良好，請繼續維持。")
        return True

def analyze_and_plot(df, analysis_category, chart_type='bar'):
    """
    依照指定類別生成總金額統計圖表，並將其保存為 BytesIO 物件。
    """
    if df.empty:
        return None, "尚無交易紀錄，無法進行分析。"

    if analysis_category not in df.columns:
        return None, f"錯誤：資料中找不到 '{analysis_category}' 類別。請檢查 Google Sheet 中的標題是否正確。"

    if '台幣金額' not in df.columns:
        return None, "錯誤：DataFrame 中缺少 '台幣金額' 欄位，無法進行分析。請確保 Google Sheet 包含此欄位且名稱正確。"

    if not pd.api.types.is_numeric_dtype(df['台幣金額']):
        return None, "錯誤：'台幣金額' 欄位不是數值型別，無法進行分析。請檢查 Google Sheet 中的資料格式。"

    summary = df.groupby(analysis_category)['台幣金額'].sum().sort_values(ascending=False)

    plt.figure(figsize=(10, 6))
    if chart_type == 'pie':
        if summary.empty or summary.sum() == 0:
            return None, "尚無資料或所有類別金額皆為0，因此無法生成圓餅圖"
        # Filter out 0 values for pie chart to avoid division by zero in autopct
        summary = summary[summary > 0]
        if summary.empty:
             return None, "尚無資料或所有類別金額皆為0，因此無法生成圓餅圖"
        plt.pie(summary, labels=summary.index, autopct='%1.1f%%', startangle=90, wedgeprops={'edgecolor': 'black'})
        plt.title(f'周邊支出(依 {analysis_category}區分(圓餅圖))', fontsize=16)
        plt.axis('equal')
    else:
        summary.plot(kind='bar', color='darkcyan')
        plt.title(f'周邊支出(依 {analysis_category}區分(長條圖))', fontsize=16)
        plt.ylabel('總金額 (NT$)', fontsize=12)
        plt.xlabel(analysis_category, fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', linestyle='--')
    plt.tight_layout()
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close()
    buf.seek(0)
    pil_image = Image.open(buf)

    summary_text = f"依{analysis_category}區分的支出總結\n" + summary.to_string(float_format="NT$ {:,.0f}".format)

    return pil_image, summary_text

def query_records(df, query_type, keyword):
    if df.empty:
        return "<p>尚無交易紀錄，無法進行查詢。</p>"

    result_df = pd.DataFrame()
    if query_type == '日期':
        if '日期' in df.columns:
            df_copy = df.copy()
            df_copy['日期'] = pd.to_datetime(df_copy['日期'], errors='coerce')
            result_df = df_copy[df_copy['日期'].dt.strftime('%Y-%m-%d').str.contains(keyword, na=False)]
    elif query_type == 'IP類別':
        if 'IP類別' in df.columns:
            result_df = df[df['IP類別'].str.contains(keyword, case=False, na=False)]
    elif query_type == '周邊類別':
        if '周邊類別' in df.columns:
            result_df = df[df['周邊類別'].str.contains(keyword, case=False, na=False)]
    elif query_type == 'IP名稱':
        if 'IP名稱' in df.columns:
            result_df = df[df['IP名稱'].str.contains(keyword, case=False, na=False)]
    else:
        return "<p>輸入無效的查詢方式。</p>"

    if result_df.empty:
        return "<p>未找到符合條件的紀錄。</p>"
    else:
        display_cols = ['日期', 'IP類別', '國家', 'IP名稱', '周邊類別', '台幣金額']
        # Filter display_cols to only include those actually present in result_df
        actual_display_cols = [col for col in display_cols if col in result_df.columns]
        if '日期' in actual_display_cols and pd.api.types.is_datetime64_any_dtype(result_df['日期']):
             result_df['日期'] = result_df['日期'].dt.strftime('%Y-%m-%d %H:%M:%S')
        html_table = result_df[actual_display_cols].to_html(index=False, float_format=lambda x: f"NT$ {x:,.0f}")
        total_amount = result_df['台幣金額'].sum()
        summary = f"<p>總計筆數: {len(result_df)} 筆, 總金額: NT$ {total_amount:,.0f}</p>"
        return html_table + summary

def ai_suggestion(df_records):
    suggestion_output = []
    suggestion_output.append("{ 財務行為分析 }")
    if df_records.empty or '台幣金額' not in df_records.columns or df_records['台幣金額'].sum() == 0:
        suggestion_output.append("紀錄不足或無有效支出金額，無法提供財務行為分析。")
    else:
        total_spending = df_records['台幣金額'].sum()
        ip_summary = df_records.groupby('IP類別')['台幣金額'].sum().sort_values(ascending=False)

        if not ip_summary.empty:
            highest_ip = ip_summary.index[0]
            highest_amount = ip_summary.iloc[0]
            percentage = (highest_amount / total_spending) * 100

            suggestion_output.append(f"您最大的支出集中在「{highest_ip}」(NT$ {highest_amount:,.0f})，佔總支出{percentage:.1f}%。")

            if percentage > 50:
                 suggestion_output.append("警示：單一目標支出佔比過高，請留意風險分散，並確保財務平衡。")
        else:
            suggestion_output.append("紀錄不足，無法提供財務行為分析。")

    suggestion_output.append("\n{ 周邊收納建議 }")

    if df_records.empty or '周邊類別' not in df_records.columns:
        suggestion_output.append("紀錄不足，無法提供周邊收納建議。")
    else:
        item_counts = df_records['周邊類別'].value_counts()

        if not item_counts.empty:
            most_bought_item = item_counts.index[0]

            advice = "建議：通用收納箱、防潮包。"

            if '徽章' in most_bought_item or '小卡' in most_bought_item:
                advice = "建議購買：A4或B5活頁收納冊、卡套、磁吸夾層板，保持乾燥。"
            elif '立牌' in most_bought_item or '吊飾' in most_bought_item:
                advice = "建議購買：分層壓克力展示架、防塵透明盒，避免陽光直射。"
            elif '同人本' in most_bought_item or '插畫集' in most_bought_item:
                advice = "建議購買：防酸書套、直立式收納盒，避免側壓造成裝幀變形。"

            suggestion_output.append(f"您最近常購買「{most_bought_item}」。")
            suggestion_output.append(f"{advice}")
        else:
            suggestion_output.append("紀錄不足，無法提供周邊收納建議。")

    return "<br>".join(suggestion_output)

def process_accounting_data_gradio(ip_category, country, ip_name, item_type, amount, currency, purchase_date, purchase_time, credentials_file, spreadsheet_name):
    if credentials_file is None:
        return "錯誤：請上傳服務帳號金鑰檔案。"
    if not spreadsheet_name:
        return "錯誤：請輸入 Google Sheet 名稱。"
    credentials_path = credentials_file.name

    try:
        worksheet, spreadsheet_id = get_google_sheet(credentials_path, spreadsheet_name)

        if worksheet is None or spreadsheet_id is None:
            return "無法連線到 Google Sheet。請檢查金鑰檔案和 Sheet 名稱。"

        # Call read_google_sheet_to_dataframe to ensure headers are fine, but discard returned df and categories
        _ = read_google_sheet_to_dataframe(credentials_path, spreadsheet_name)

        result = accountingInCategory(
            ip_category=ip_category,
            country=country,
            ip_name=ip_name,
            item_type=item_type,
            amount=amount,
            currency=currency,
            purchase_date=purchase_date,
            purchase_time=purchase_time,
            worksheet=worksheet
        )
        google_sheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/edit"
        success_message = (
            f"記帳成功！已新增記錄：{result['IP類別']} - {result['IP名稱']} - {result['周邊類別']}，\n"
            f"台幣金額：{result['台幣金額']:.2f}<br>\n"
            f"<a href='{google_sheet_url}' target='_blank'>點擊此處查看您的 Google Sheet</a>"
        )
        return success_message

    except Exception as e:
        return f"處理記帳時發生錯誤：{e}"

ip_category_input_tab1 = gr.Textbox(label="IP類別 (例如: VTuber)")
country_input_tab1 = gr.Textbox(label="國家 (例如: 日本)")
ip_name_input_tab1 = gr.Textbox(label="IP名稱 (例如: Mio、Hololive)")
item_type_input_tab1 = gr.Textbox(label="周邊類別 (例如: 徽章)")
amount_input_tab1 = gr.Number(label="金額")
currency_input_tab1 = gr.Dropdown(choices=["TWD", "USD", "JPY", "KRW", "EUR"], label="幣別")
purchase_date_input_tab1 = gr.Textbox(label="購買日期 (YYYY-MM-DD)", value=datetime.date.today().strftime("%Y-%m-%d"))
purchase_time_input_tab1 = gr.Textbox(label="購買時間 (HH:MM:SS)", value=datetime.datetime.now().strftime("%H:%M:%S"))
credentials_file_input_tab1 = gr.File(label="服務帳號金鑰檔案 (.json)", file_types=[".json"]) # Re-added file input
spreadsheet_name_input_tab1 = gr.Textbox(label="Google Sheet 名稱")
output_tab1 = gr.HTML(label="操作結果")

accounting_interface = gr.Interface(
    fn=process_accounting_data_gradio,
    inputs=[
        ip_category_input_tab1,
        country_input_tab1,
        ip_name_input_tab1,
        item_type_input_tab1,
        amount_input_tab1,
        currency_input_tab1,
        purchase_date_input_tab1,
        purchase_time_input_tab1,
        credentials_file_input_tab1, # Added to inputs
        spreadsheet_name_input_tab1
    ],
    outputs=output_tab1,
    title="周邊商品記帳",
    flagging_mode="manual"
)

def process_analysis_data_gradio(credentials_file, spreadsheet_name, analysis_category, chart_type):
    if credentials_file is None:
        return None, "錯誤：請上傳服務帳號金鑰檔案。"
    if not spreadsheet_name:
        return None, "錯誤：請輸入 Google Sheet 名稱。"
    credentials_path = credentials_file.name

    try:
        df_records, _, _, _, _ = read_google_sheet_to_dataframe(credentials_path, spreadsheet_name)

        if df_records.empty:
            return None, "尚無交易紀錄，無法進行分析。"
        plot_bytes, plot_summary_text = analyze_and_plot(df_records, analysis_category, chart_type)
        ai_suggestions_text = ai_suggestion(df_records)
        final_output_text = f"#### 財務分析報告 ####\n{plot_summary_text}\n\n#### AI 建議 ####\n{ai_suggestions_text}"

        return plot_bytes, final_output_text

    except Exception as e:
        return None, f"處理分析時發生錯誤：{type(e).__name__}: {e}"

credentials_file_input_tab2 = gr.File(label="服務帳號金鑰檔案 (.json)", file_types=[".json"]) # Re-added file input
spreadsheet_name_input_tab2 = gr.Textbox(label="Google Sheet 名稱")
analysis_category_input_tab2 = gr.Dropdown(
    choices=['IP類別', '國家', 'IP名稱', '周邊類別', '月份'],
    label="選擇分析維度",
    value='IP類別'
)
chart_type_input_tab2 = gr.Dropdown(
    choices=['bar', 'pie'],
    label="選擇圖表類型",
    value='bar'
)

output_plot_tab2 = gr.Image(label="分析圖表", type="pil", interactive=False, height=400)
output_text_tab2 = gr.HTML(label="分析總結與AI建議")

analysis_interface = gr.Interface(
    fn=process_analysis_data_gradio,
    inputs=[
        credentials_file_input_tab2, # Added to inputs
        spreadsheet_name_input_tab2,
        analysis_category_input_tab2,
        chart_type_input_tab2
    ],
    outputs=[
        output_plot_tab2,
        output_text_tab2
    ],
    title="財務分析與AI建議",
    flagging_mode="manual"
)

def process_query_data_gradio(credentials_file, spreadsheet_name, mode, query_type, keyword, analysis_category_for_queried_data, chart_type_for_queried_data):
    output_table = ""
    output_plot = None
    output_analysis_text = ""

    if credentials_file is None:
        return "<p style='color:red;'>錯誤：請上傳服務帳號金鑰檔案。</p>", None, ""
    if not spreadsheet_name:
        return "<p style='color:red;'>錯誤：請輸入 Google Sheet 名稱。</p>", None, ""
    credentials_path = credentials_file.name

    try:
        df_records, _, _, _, _ = read_google_sheet_to_dataframe(credentials_path, spreadsheet_name)
        if df_records.empty:
            return "<p>尚無交易紀錄，無法進行查詢。</p>", None, ""
        df_queried_records = pd.DataFrame()
        if query_type == '日期':
            if '日期' in df_records.columns:
                df_copy = df_records.copy()
                df_copy['日期'] = pd.to_datetime(df_copy['日期'], errors='coerce')
                df_queried_records = df_copy[df_copy['日期'].dt.strftime('%Y-%m-%d').str.contains(keyword, na=False)]
        elif query_type == 'IP類別':
            if 'IP類別' in df_records.columns:
                df_queried_records = df_records[df_records['IP類別'].str.contains(keyword, case=False, na=False)]
        elif query_type == '周邊類別':
            if '周邊類別' in df_records.columns:
                df_queried_records = df_records[df_records['周邊類別'].str.contains(keyword, case=False, na=False)]
        elif query_type == 'IP名稱':
            if 'IP名稱' in df_records.columns:
                df_queried_records = df_records[df_records['IP名稱'].str.contains(keyword, case=False, na=False)]
        else:
            output_table = "<p>輸入無效的查詢方式。</p>"
            return output_table, output_plot, output_analysis_text

        if mode == '顯示紀錄':
            output_table = query_records(df_queried_records, query_type, keyword)
            output_plot = None
            output_analysis_text = ""

        elif mode == '分析查詢結果':
            if df_queried_records.empty:
                output_analysis_text = "<p>查詢結果為空，無法進行分析。</p>"
                output_table = ""
                output_plot = None
            else:
                plot_bytes, plot_summary_text = analyze_and_plot(df_queried_records, analysis_category_for_queried_data, chart_type_for_queried_data)
                output_plot = plot_bytes

                ai_suggestions_text = ai_suggestion(df_queried_records)

                output_analysis_text = f"#### 查詢結果分析報告 ####\n{plot_summary_text}\n\n#### 查詢結果AI建議 ####\n{ai_suggestions_text}"
                output_table = ""

        return output_table, output_plot, output_analysis_text

    except Exception as e:
        return f"<p style='color:red;'>處理查詢時發生錯誤：{type(e).__name__}: {e}</p>", None, ""

credentials_file_input_tab3 = gr.File(label="服務帳號金鑰檔案 (.json)", file_types=[".json"]) # Re-added file input
sheet_name_input_tab3 = gr.Textbox(label="Google Sheet 名稱")
mode_input_tab3 = gr.Dropdown(
    choices=['顯示紀錄', '分析查詢結果'],
    label="選擇操作模式",
    value='顯示紀錄'
)
query_type_input_tab3 = gr.Dropdown(
    choices=['日期', 'IP類別', 'IP名稱', '周邊類別'],
    label="選擇查詢維度",
    value='IP類別'
)
keyword_input_tab3 = gr.Textbox(label="查詢關鍵字")
analysis_category_for_queried_data_input_tab3 = gr.Dropdown(
    choices=['IP類別', '國家',  'IP名稱', '周邊類別', '月份'],
    label="分析查詢結果的維度",
    value='IP類別',
    visible=False
)
chart_type_for_queried_data_input_tab3 = gr.Dropdown(
    choices=['bar', 'pie'],
    label="分析查詢結果的圖表類型",
    value='bar',
    visible=False
)
output_query_table_tab3 = gr.HTML(label="查詢結果表格")
output_query_plot_tab3 = gr.Image(label="查詢結果圖表", type="pil", interactive=False, height=400, visible=False)
output_query_analysis_text_tab3 = gr.HTML(label="查詢結果分析與AI建議", visible=False)

query_interface = gr.Interface(
    fn=process_query_data_gradio,
    inputs=[
        credentials_file_input_tab3, # Added to inputs
        sheet_name_input_tab3,
        mode_input_tab3,
        query_type_input_tab3,
        keyword_input_tab3,
        analysis_category_for_queried_data_input_tab3,
        chart_type_for_queried_data_input_tab3
    ],
    outputs=[
        output_query_table_tab3,
        output_query_plot_tab3,
        output_query_analysis_text_tab3
    ],
    title="查詢交易紀錄",
    flagging_mode="manual"
)

def update_query_visibility(mode):
    # Fix typo: true should be True
    if mode == '顯示紀錄':
        return gr.update(visible=True), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)
    elif mode == '分析查詢結果':
        return gr.update(visible=False), gr.update(visible=True), gr.update(visible=True), gr.update(visible=True), gr.update(visible=True), gr.update(visible=True) # Keyword input should remain visible for filtering
    return gr.update(visible=True), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)

with gr.Blocks(title="周邊商品記帳系統") as demo:
    gr.Markdown("# 周邊商品記帳系統")
    with gr.Tabs() as tabs:
        with gr.TabItem("記帳", id=0):
            accounting_interface.render()
        with gr.TabItem("分析財務&AI分析", id=1):
            analysis_interface.render()
        with gr.TabItem("查詢", id=2):
            with gr.Column():
                gr.Markdown("查詢交易紀錄與分析")
                with gr.Row():
                    credentials_file_input_tab3.render() # Render credentials file input
                    sheet_name_input_tab3.render()
                mode_input_tab3.render()
                with gr.Row():
                    query_type_input_tab3.render()
                    keyword_input_tab3.render()
                analysis_category_for_queried_data_input_tab3.render()
                chart_type_for_queried_data_input_tab3.render()
                query_button = gr.Button("執行查詢/分析")
                output_query_table_tab3.render()
                output_query_plot_tab3.render()
                output_query_analysis_text_tab3.render()
            mode_input_tab3.change(
                update_query_visibility,
                inputs=[mode_input_tab3],
                outputs=[
                    output_query_table_tab3,
                    output_query_plot_tab3,
                    output_query_analysis_text_tab3,
                    analysis_category_for_queried_data_input_tab3,
                    keyword_input_tab3, # Keyword input visibility needs to be updated
                    chart_type_for_queried_data_input_tab3
                ]
            )

            query_button.click(
                process_query_data_gradio,
                inputs=[
                    credentials_file_input_tab3, # Pass credentials file input
                    sheet_name_input_tab3,
                    mode_input_tab3,
                    query_type_input_tab3,
                    keyword_input_tab3,
                    analysis_category_for_queried_data_input_tab3,
                    chart_type_for_queried_data_input_tab3
                ],
                outputs=[
                    output_query_table_tab3,
                    output_query_plot_tab3,
                    output_query_analysis_text_tab3
                ]
            )

demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2cb7d6a38f719dfc03.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


成功連線到 Google Sheet: '測試', 工作表: '工作表1'
成功連線到 Google Sheet: '測試', 工作表: '工作表1'

GoogleSheet讀取的DataFrame資訊
DataFrame Columns: ['日期', 'IP類別', '國家', 'IP名稱', '周邊類別', '金額', '幣別', '台幣金額', '時間', '月份']
DataFrame Head:
           日期 IP類別  國家  IP名稱 周邊類別   金額   幣別  台幣金額        時間       月份
0 2025-12-16   漫畫  日本   神樂鉢   立牌  500  TWD   500  05:18:59  2025-12
1 2025-12-16   漫畫  日本    死神  寶寶娃  500  TWD   500  05:33:56  2025-12
2 2025-12-16   動畫  日本    死神   徽章  500  JPY   110  05:33:56  2025-12
3 2025-12-16   漫畫  日本   神樂鉢   立牌  800  TWD   800  13:29:13  2025-12
4 2025-12-16   動畫  日本  失憶投捕   立牌  800  TWD   800  15:57:43  2025-12
-------------------------------------------
轉換結果: 120 TWD = 120.00 TWD

已準備好以下記錄：{'日期': '2025-12-17', 'IP類別': '漫畫', '國家': '日本', 'IP名稱': '失憶投捕', '周邊類別': '漫畫單行本', '金額': 120, '幣別': 'TWD', '台幣金額': 120.0, '時間': '17:48:25'}
資料已成功寫入 Google Sheet。
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8cb8d8c3a7202f0ee6.gradio.live
Killing tunnel 1